# Is my beer brewing at the right temperature?

## Problem Statement:
During lockdown 2.0 I've been brewing my own beer, but the beer must be kept between 20 and 25 degrees celcius in order for the fermentation to proceed optimally. Too warm and the yeast risks dying, too cold and no fermentation will happen at all!

Therefore I needed a way to find out the temperature of the beer, and ideally one that had a level of automation involved in it. Enter my trusty raspberryPi!


## Methodology:
I set up a DS18B20 One wire digital thermometer to read the ambient temperature in both degrees celcius and farenheit and then connected to the output of this using a dataplicity account. This allowed me to control the rPi and the thermometer remotely. From here the temperature was displayed on a webpage, 

## Version 1:
- Connects to the rPi
- Records readings at a fixed sampling rate (1 every 10 seconds)
- Saves these results after a specified number of iterations

## Future Versions:
- Can run at a specified time 
- Can allow a notification to be given if the temperature drops below 20 oC for a certain amount of time

In [1]:
# Imports packages
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
from apscheduler.schedulers.blocking import BlockingScheduler

In [2]:
#Initialises data storage
data=[]
#Parameters
timebetween=10
number_of_iterations=10

for i in range (0,number_of_iterations):
    #Starts timing loop
    start = time.time()
    now=datetime.now()
    #Gets data from dataplicity connection
    response = requests.get("https://censual-mosquito-5256.dataplicity.io/")
    content = response.content
    #Parses html response
    parser = BeautifulSoup(content, 'html.parser')
    tempraw=parser.text
    #Cleans result
    tempclean=tempraw.replace("\n",'').replace("'",'').replace('"','').replace(':','').replace(',','')
    lst=tempclean.split(" ")
    #Stores current time and the temperature in degrees celcius 
    data.append([now, float(lst[3])])
    #Ends the timing loop
    end = time.time()
    #Calculates the time taken and uses it to keep ~10 seconds between iterations
    time.sleep(timebetween-(end-start))

#Stores the data in a dataframe
tempdata=pd.DataFrame(data)
#Ammends column names
tempdata.columns=['Time', 'Temperature / oC']
print(tempdata)
#Saves the data to a csv file
tempdata.to_csv ('tempdata_12Nov.csv', index = False, header=True)

                        Time  Temperature / oC
0 2020-11-12 12:16:42.849571              21.4
1 2020-11-12 12:16:53.201163              24.8
2 2020-11-12 12:17:02.599700              24.9
3 2020-11-12 12:17:12.350258              25.7
4 2020-11-12 12:17:22.374831              25.9
5 2020-11-12 12:17:32.540413              26.9
6 2020-11-12 12:17:42.752997              26.8
7 2020-11-12 12:17:52.371547              26.0
8 2020-11-12 12:18:02.333117              25.3
9 2020-11-12 12:18:12.652707              24.6
